In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

import collections
def bfs(C, F, l, src, snk) :
  V = len(C)
  Q = collections.deque()
  Q.append(src)
  l[src] = 0
  while Q :
    u = Q.popleft()
    for v in range(V) :
      if l[v] == -1 and C[u][v] - F[u][v] > 0 :
        l[v] = l[u] + 1
        Q.append(v)
  return l[snk] != -1

def dfs(C, F, W, l, u, snk, cur = 1 << 63) :
  if u == snk : return cur
  V = len(C)
  while W[u] < V :
    v = W[u]
    if l[v] == l[u] + 1 and C[u][v] > F[u][v] :
      A = min(cur, C[u][v] - F[u][v])
      tmp = dfs(C, F, W, l, v, snk, A)
      if tmp > 0 :
        F[u][v] += tmp
        F[v][u] -= tmp
        return tmp
    W[u] += 1
  return 0

def dinic(C, src, snk) :
  V = len(C)
  F = [[0] * V for _ in range(V)]
  ret = 0

  while True :
    l = [-1] * V
    W = [0] * V
    if not bfs(C, F, l, src, snk) : break
    while True:
      tmp = dfs(C, F, W, l, src, snk)
      if tmp == 0 : break
      ret += tmp
  return ret, F

def sol():
  N = int(input())
  L1 = [*map(int, input().split())]
  L2 = [*map(int, input().split())]
  
  V = N * 2 + 2
  src = V - 2
  snk = V - 1
  C = [[0] * V for _ in range(V)]
  for i in range(N) :
    C[src][i] = L1[i]
    C[N+i][snk] = L2[i]

  for i in range(N) :
    for j in range(N) :
      C[i][N+j] = 1

  res, F = dinic(C, src, snk)
  ans = [[0] * N for _ in range(N)]
  for i in range(N) :
    for j in range(N) :
      if F[i][N+j] :
        ans[i][j] = 1

  for i, l in enumerate(ans) :
    sum = 0
    for j, v in enumerate(l) :
      sum += v
    if sum != L1[i] : break
  
  for i, l in enumerate(zip(*ans)) :
    sum = 0
    for j, v in enumerate(l) :
      sum += v
    if sum != L2[i] : break
  else :
    sys.stdout.write("1\n")
    for l in ans :
      sys.stdout.write("".join(map(str, l)) + "\n")
    return
  return sys.stdout.write("-1")

sol()

### 풀이
- 정점을 행과 열의 번호 기준으로 나누고 행으로 가는 간선의 용량을 각 (행/열에 대한)두줄의 입력에 맞게 설정하고 i행 -> j열로 가는 간선을 1로 설정한뒤 최대유량을 흘린다.
- 그리고 i행 -> j열로 가는 유량이 있을 경우 ans[i][j]에 1을 넣는다.
- ans의 각 행과 열의 합이 입력 두줄의 합과 맞는지 검증한다.